In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes,svm
from sklearn.metrics import accuracy_score

C:\Users\kelse\Anaconda3\new\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\Users\kelse\Anaconda3\new\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https:/

In [2]:
#Set Random seed
np.random.seed(500)

# Add the Data using pandas
df = pd.read_csv(r"C:/Users/kelse/OneDrive/Documents/UOM-Imp Info/FYP/HTML-elements.csv",encoding = "latin-1")

df = df[df.Category != 'Check Out']
df.dropna
#Corpu.to_excel(r"C:/Users/kelse/OneDrive/Documents/UOM-Imp Info/FYP/AddToCartvsCheckout.xlsx", encoding='utf-8',index=False)

<bound method DataFrame.dropna of                    websites_link  \
0        https://www.amazon.com/   
1          https://www.ebay.com/   
2          https://shop.bbc.com/   
3    https://www.aliexpress.com/   
4      https://www.amazon.co.uk/   
..                           ...   
271                    kohls.com   
272                 digikala.com   
273                      otto.de   
274                cdiscount.com   
275             ticketmaster.com   

                                           HTMLElement     Category  
0    <input id="twotabsearchtextbox" value="" name=...       Search  
1    <input class="gh-tb ui-autocomplete-input" ari...       Search  
2    <input type="search" id="bc-sf-search-box-0" c...       Search  
3    <input type="text" placeholder="iphone 13 pro ...       Search  
4    <input id="twotabsearchtextbox" value="" name=...       Search  
..                                                 ...          ...  
271  <input type="button" title="Add To Car

In [ ]:
#Corpus = pd.read_excel(r"C:/Users/kelse/OneDrive/Documents/UOM-Imp Info/FYP/AddToCartvsCheckout.xlsx")
#Corpus

In [3]:
# Step - 1: Data Pre-processing - This will help in getting better results through the classification algorithms
df = df.astype(str)
# Step - 1a : Remove blank rows if any.
df['HTMLElement'].dropna(inplace=True)

# Step - 1b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
df['HTMLElement'] = [entry.lower() for entry in df['HTMLElement']]

# Step - 1c : Tokenization : In this each entry in the corpus will be broken into set of words
df['HTMLElement']= [word_tokenize(entry) for entry in df['HTMLElement']]

# Step - 1d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.

# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV


for index,entry in enumerate(df['HTMLElement']):
    # Declaring Empty List to store the words that follow the rules for this step
    processed_text = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            processed_text.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    df.loc[index,'processed_text'] = str(processed_text)

print(df['processed_text'].head())

# Step - 2: Split the model into Train and Test Data set
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(df['processed_text'],df['Category'],test_size=0.3)

# Step - 3: Label encode the target variable  - This is done to transform Categorical data of string type in the data set into numerical values
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y.astype(str))
Test_Y = Encoder.fit_transform(Test_Y.astype(str))

Test_Y


0    ['input', 'twotabsearchtextbox', 'auto', 'sear...
1    ['input', 'list', 'false', 'search', 'anything...
2           ['input', 'search', 'label', 'q', 'input']
3    ['input', 'text', 'iphone', 'pro', 'max', 'cas...
4    ['input', 'twotabsearchtextbox', 'auto', 'sear...
Name: processed_text, dtype: object


array([0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1])

In [4]:
print(df['Category'].head())

0    Search
1    Search
2    Search
3    Search
4    Search
Name: Category, dtype: object


In [6]:
import pickle 

X, y = df['processed_text'], df['Category'].astype(str)
#data_count, batch_count = X.shape
#X=np.reshape(X, (data_count*batch_count))
#y = np.reshape(y , (data_count*batch_count, -1))
#metrics = []

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
import joblib

skf = StratifiedKFold(n_splits=5)
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    #print("TRAIN:", train_index, "TEST:", test_index)
    
    vect = CountVectorizer(ngram_range=(1,2), max_features=1000 , stop_words="english")
    X_train_bow = vect.fit_transform(X_train)
    X_test_bow = vect.transform(X_test)
    #Tfidf_vect = TfidfVectorizer(max_features=5000)
    #Tfidf_vect.fit(df['text_final'])
    #Train_X_Tfidf = Tfidf_vect.transform(X_train)
    #Test_X_Tfidf = Tfidf_vect.transform(X_test)
    #nb = naive_bayes.MultinomialNB()
    #nb.fit(X_train_bow, y_train)
    SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
    SVM.fit(X_train_bow,y_train)
    predictions_SVM = SVM.predict(X_test_bow)
    # save the model to disk
    pickle.dump(SVM, open('C:/Users/kelse/OneDrive/Documents/UOM-Imp Info/FYP/finalized_svm_model.pickle', 'wb'))
    pickle.dump(vect, open("C:/Users/kelse/OneDrive/Documents/UOM-Imp Info/FYP/vector.pickel", "wb"))

    
    
    

print("SVM Accuracy Score -> ",accuracy_score(y_test,predictions_SVM)*100)
print("Precision:",metrics.precision_score(y_test, predictions_SVM,average="binary",pos_label="Add To Cart"))
print("Recall:",metrics.recall_score(y_test, predictions_SVM, average="binary",pos_label="Add To Cart"))

C:\Users\kelse\Anaconda3\new\lib\site-packages\sklearn\model_selection\_split.py:670: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_folds = np.zeros(n_samples, dtype=np.int)
C:\Users\kelse\Anaconda3\new\lib\site-packages\sklearn\model_selection\_split.py:442: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish t

SVM Accuracy Score ->  98.18181818181819
Precision: 0.9629629629629629
Recall: 1.0


In [ ]:
X, y = df['processed_text'], df['Category'].astype(str)
#data_count, batch_count = X.shape
#X=np.reshape(X, (data_count*batch_count))
#y = np.reshape(y , (data_count*batch_count, -1))
#metrics = []

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics

skf = StratifiedKFold(n_splits=5)
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    #print("TRAIN:", train_index, "TEST:", test_index)
    
    #using BagOfWords
    vect = CountVectorizer(ngram_range=(1,2), max_features=1000 , stop_words="english")
    X_train_bow = vect.fit_transform(X_train)
    X_test_bow = vect.transform(X_test)
    
    #Using TF-IDF
    #vectorizer = TfidfVectorizer(max_features=5000)
    #vectorizer.fit(Corpus['text_final'])
    #X_train_tfidf = vectorizer.transform(X_train)
    #X_test_tfidf = vectorizer.transform(X_test)
    
    nb = naive_bayes.MultinomialNB()
    nb.fit(X_train_bow, y_train)
    predictions_NB = nb.predict(X_test_bow)
    

#metrics = np.array(metrics)
#print('Mean accuracy: ', np.mean(metrics, axis=0))
#print('Std for accuracy: ', np.std(metrics, axis=0))

print("Naive Bayes Accuracy Score -> ",accuracy_score(y_test,predictions_NB)*100)
print("Precision:",metrics.precision_score(y_test, predictions_NB,average="binary",pos_label="Add To Cart"))
print("Recall:",metrics.recall_score(y_test, predictions_NB,average="binary",pos_label="Add To Cart"))

In [ ]:
##Confusion Matrix
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

#Generate the confusion matrix
cf_matrix = confusion_matrix(Test_Y, predictions_SVM)

print(cf_matrix)

import seaborn as sns

ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues')

ax.set_title('Confusion Matrix with labels\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])

## Display the visualization of the Confusion Matrix.
plt.show()

print (classification_report(y_test, predictions_SVM))

In [ ]:
def preprocess_text(text):

    stop_words = set(stopwords.words("english"))
    element = [entry.lower() for entry in text]
    
    element= [word_tokenize(entry) for entry in text] #tokenization
    lemmatizer = WordNetLemmatizer()

    final_words = []
    for word in element:
        if word not in stopwords.words('english'):
            final_words.append(word)
            
    text_processed =[ ' '.join([lemmatizer.lemmatize(word) for word_list in element for word in word_list])]
    
    
    return text_processed

In [ ]:
test_Search = ['<input class="quer" type="text" value="" style="border: solid #b9b9b9 2px ; height: 3em; width: 78%; border-radius:10px; padding-left: 8%; font-size: 1.5em; font-weight: 600; outline: none; margin-top: 2%" placeholder="Search">']
test_AddToCart = ['<input type="button" id="add-to-cart-button-38224" value="Add to cart" class="button-1 add-to-cart-button nopAjaxCartProductVariantAddToCartButton" data-productid="38224">']
#test_Checkout = ['<button type="button" class="btnTemplateCheckOut" data-toggle="modal" data-target="#exampleModalLong">Secure CheckOut</button>']
text = preprocess_text(test_Search)
test_input = vect.transform(text)
test_input.shape
print(test_Search)

In [ ]:
#Testing the model 
res= SVM.predict(test_input)[0]
print(res,text)
if res==1:
    print("Search")
elif res==0:
    print("AddToCart")

In [ ]:
print(type(test_Search))

In [13]:
import uvicorn
from os.path import dirname, join, realpath
import joblib
from typing import List
from typing import Optional
from fastapi import FastAPI, Query
from fastapi import FastAPI, Request
from fastapi.responses import JSONResponse


app = FastAPI(
    title="HTML Element Classifier Model API",
    description="API for ML model",
    version="0.1",
)

# load model
#with open(
#    join(dirname(realpath('C:/Users/kelse/OneDrive/Documents/UOM-Imp Info/FYP/')), "finalized_svm_model.pkl"), "rb"
#) as f:
 #   model = joblib.load(f)

model=joblib.load('C:/Users/kelse/OneDrive/Documents/UOM-Imp Info/FYP/finalized_svm_model.pickle')
    
def preprocess_text(text):

    stop_words = set(stopwords.words("english"))
    element = [entry.lower() for entry in text]
    
    #Step - 1c : Tokenization : In this each entry in the corpus will be broken into set of words
    element= [word_tokenize(entry) for entry in text]
    lemmatizer = WordNetLemmatizer()
    
    final_words = []
    for word in element:
        if word not in stopwords.words('english'):
            final_words.append(word)
            
    text_processed =[ ' '.join([lemmatizer.lemmatize(word) for word_list in element for word in word_list])]
    
    return text_processed


@app.get("/predict-element")
def predict(element: Optional[List[str]] = Query(["<input></input>"])):
    
    ##Pre-process text
    processed_text = preprocess_text(element)
   
   # vect = CountVectorizer(decode_error=" Replace ", vocabulary=p.load(open("C:/Users/kelse/OneDrive/Documents/UOM-Imp Info/FYP/vector.pickel", "rb")))
    vect = CountVectorizer(ngram_range=(1,2), max_features=1000 , stop_words="english")
    test = vect.transform(processed_text)
    
   
    prediction = model.predict(test)[0]
    output = int(prediction[0])
   
    
    # output dictionary
    pred = {0: "AddToCart", 1: "Search"}
    
    # show results
    result = {"prediction": pred[prediction]}

    return result

@app.exception_handler(ValueError)
async def value_error_exception_handler(request: Request, exc: ValueError):
    return JSONResponse(
        status_code=400,
        content={"message": str(exc)},
    )

In [ ]:
from pyngrok import ngrok

ngrok_tunnel = ngrok.connect(8000)

ngrok_tunnel

import nest_asyncio
import uvicorn

nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [18724]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:10479 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:10479 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:10484 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:10484 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:10484 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:10506 - "GET /predict-element?element=%3C%20input%20class%3D%20%27%27%20quer%20%27%27%20type%3D%20%27%27%20text%20%27%27%20value%3D%20%27%27%20%27%27%20style%3D%20%27%27%20border%20%3A%20solid%20%23%20b9b9b9%202px%20%3B%20height%20%3A%203em%20%3B%20width%20%3A%2078%20%25%20%3B%20border-radius%3A10px%20%3B%20padding-left%20%3A%208%20%25%20%3B%20font-size%20%3A%201.5em%20%3B%20font-weight%20%3A%20600%20%3B%20outline%20%3A%20none%20%3B%20margin-top%20%3A%202%20%25%20%27%27%20placeholder%3D%20%27%27%20Search%20%27%27%20%3E HTTP/1.1" 400 Bad Request
